In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
reports_url = 'https://www.gunviolencearchive.org/mass-shooting'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
reports_page = requests.get(reports_url, headers = headers)
reports_page

<Response [200]>

In [3]:
reports_page_soup = BeautifulSoup(reports_page.text, 'html.parser')
reports_list = reports_page_soup.find_all(name="table")

In [4]:
years = {2014:10, 2015:13 ,2016:15 , 2017:13 , 2018:13 ,2019:14}

In [5]:
reports_list[:5]

[<table class="responsive sticky-enabled">
 <thead><tr><th column="Base.IncidentID"><a class="active" href="/mass-shooting?sort=asc&amp;order=Incident%20ID" title="sort by Incident ID">Incident ID</a></th><th class="active" column="Base.IncidentDate"><a class="active" href="/mass-shooting?sort=asc&amp;order=Incident%20Date" title="sort by Incident Date">Incident Date<img alt="sort ascending" height="13" src="https://www.gunviolencearchive.org/misc/arrow-asc.png" title="sort ascending" typeof="foaf:Image" width="13"/></a></th><th column="Location.State"><a class="active" href="/mass-shooting?sort=asc&amp;order=State" title="sort by State">State</a></th><th column="Location.CityOrCounty"><a class="active" href="/mass-shooting?sort=asc&amp;order=City%20Or%20County" title="sort by City Or County">City Or County</a></th><th column="Location.Address"><a class="active" href="/mass-shooting?sort=asc&amp;order=Address" title="sort by Address">Address</a></th><th column="Counts.NumberOfParticipa

In [6]:
def level_one_scraper(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    reports_page = requests.get(url, headers = headers)
    return pd.read_html(reports_page.text)

In [7]:
def page0_scraper(df, year):
    url = 'https://www.gunviolencearchive.org/mass-shooting?year=' + str(year)
    df1 = level_one_scraper(url)
    df = df.append(df1[0], True)
    return df

def other_page_scraper(df, year, pages):
    for i in range(1, pages + 1):
        url = 'https://www.gunviolencearchive.org/mass-shooting?page='+ str(i)+'&year=' + str(year)
        df1 = level_one_scraper(url)
        df = df.append(df1[0], True)
        time.sleep(0.5)
    return df

In [8]:
def scraper():
    df = pd.DataFrame()
    for year, pages in years.items():
        df = page0_scraper(df, year)
        df = other_page_scraper(df, year, pages)
    return df

In [9]:
df = scraper()